# NTLK Quetes 4  Bag of words

* 1 Importe cet ensemble de données de tweets dans un DataFrame.
* 2 Ne garde que les tweets positifs et négatifs (tu excluras donc les neutral). Quel est le pourcentage de tweets positifs/négatifs ?
* 3 Copie la colonne text dans une Série X, et la colonne sentiment dans une Série y. Applique un train test split avec le random_state = 32 et un train_size de 0.75.
* 4 Crée un modèle vectorizer avec scikit-learn en utilisant la méthode Countvectorizer. Entraîne ton modèle sur X_train, puis crée une matrice de features X_train_CV. Crée la matrice X_test_CV sans ré-entraîner le modèle. Le format de la matrice X_test_CV doit être 4091x15806 avec 44633 stored elements.
* 5 Entraîne maintenant une régression logistique avec les paramètres par défaut. Tu devrais obtenir les résultats suivants : 0.966 pour le test d'entraînement, et * 0.877 pour l'ensemble de test.
*  6 Étape bonus : essaye d'afficher 10 tweets qui ont été mal prédits (faux positifs ou faux négatifs). Aurais-tu fait mieux que l'algorithme ?

## imports

In [1]:
import os
import io
import nltk
import gzip
import spacy
import string
import random
import secrets
import datetime
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import svm
import plotly.io as pio
from sklearn import tree
from typing import Counter
import plotly.express as px
from fuzzywuzzy import fuzz
from textblob import TextBlob
from joblib import dump, load
from bs4 import BeautifulSoup
import category_encoders as ce
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import plotly.graph_objects as go
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy
from nltk.stem import PorterStemmer
from nltk.probability import FreqDist
from nltk.stem import SnowballStemmer
from sklearn.decomposition import PCA
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import silhouette_score
from sklearn.ensemble import IsolationForest
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import AgglomerativeClustering
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score
from scipy.spatial.distance import pdist, squareform
from textblob_fr import PatternTagger, PatternAnalyzer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.feature_extraction.text import CountVectorizer
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from flask import Flask, request, render_template, session, url_for, redirect
from sklearn.preprocessing import (MaxAbsScaler, MinMaxScaler, Normalizer,
                                   PowerTransformer, QuantileTransformer, RobustScaler, StandardScaler)
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, TargetEncoder


c:\Users\romar\AppData\Local\Programs\Python\Python312\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Conseils
* Train test split
* Fit et transform du Vectorizer sur le train set
* Fit et score du classifier sur le train set
* Transform (sans fit !) du Vectorizer sur le test set
* Predict et score du classifier sur le test set

### Paramètres du CountVectorizer

A l'initialisation du CountVectorizer, tu peux spécifier quelques paramètres très intéressants. Citons notamment :

* lowercase : permet de convertir tout le texte en minuscule
* stop_words : permet de spécifier une liste de stopwords, qui ne généreront donc pas de colonnes dédiées
* ngram_range : permet de spécifier si des bigrammes ou n-grammes doivent être pris en compte
* max_features : limite le nombre de mots maximum, en ne prenant que les mots les plus fréquents


In [17]:
# fichier_to_test = "./le_bonheur.txt"
# print(f"\n fichier_to_test :\n{fichier_to_test} \n")

# # Lire le fichier
# with open(fichier_to_test, "r", encoding="utf-8") as file:
#     texte = file.read()

# # Tokenisation par phrase, liste les phrases dans le texte
# token_phrases = nltk.sent_tokenize(texte)
# print(f"\n token_phrases:\n{token_phrases} \n")

# Charger le DataFrame à partir du fichier CSV
df = pd.read_csv("tweets_train.csv")
# on ne garde pas les lignes \n\n
# df = df['sentiment']
# tokens = df['tokens'].tolist()

# # transformation en matrice creuse
# vectorizer = CountVectorizer()
# matrice_creuse=vectorizer.fit_transform(df['sentiment'])
# print(f"\n matrice_creuse:\n{matrice_creuse.shape[1]} \n")
# # afficher la liste des noms de colonnes
# liste_colonne_matrice_creuse=vectorizer.get_feature_names_out()
# print(f"\n liste_colonne_matrice_creuse:\n{liste_colonne_matrice_creuse} \n")



 matrice_creuse:
3 


 liste_colonne_matrice_creuse:
['negative' 'neutral' 'positive'] 



1. Filtrer les Tweets Positifs et Négatifs
['negative' 'neutral' 'positive'] 


In [30]:
# Charger le DataFrame à partir du fichier CSV
df = pd.read_csv("tweets_train.csv")
print(f"\n df.info():\n{df.info()} \n")
# on ne garde pas les lignes \n\n
#  Filtre les tewets positifs et negatifs
df_filtered = df[df['sentiment'].isin(['positive', 'negative'])]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27480 entries, 0 to 27479
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27480 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27480 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB

 df.info():
None 



2. Calculer le Pourcentage de Tweets Positifs/Négatifs

In [32]:
pourcentage_positif = (df_filtered['sentiment'] == 'positive').mean()
pourcentage_negatif = (df_filtered['sentiment'] == 'negative').mean()
print(f"\n pourcentage_positif:\n{pourcentage_positif:.2f} \n")
print(f"\n pourcentage_negatif:\n{pourcentage_negatif:.2f} \n")



 pourcentage_positif:
0.52 


 pourcentage_negatif:
0.48 



3. Préparation des Données pour le Modèle

In [33]:
X = df_filtered['text']
y = df_filtered['sentiment']


4. Split Train/Test

In [59]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.75, random_state=32)
# Vérifier la dimension de X_test_CV
print(f"\n X_test_CV.shape:\n{X_test.shape} \n")
print(f"\n X_test_CV.size:\n{X_test.info} \n")



 X_test_CV.shape:
(4091,) 


 X_test_CV.size:
<bound method Series.info of 5680      - no,  is buttfuck stupid. I`m just silly and...
7661      get better omg i still dont believe that i di...
2670     HollowbabesHere comes the utter shite #bgt <I ...
5020      Thank You Clayton. Going to my favorite Greek...
26962     I`m watching it at the moment  -sighs- and st...
                               ...                        
4062                                       I can`t take it
4618      so where r u spinning now that the Hookah is ...
18293              WHAT?! i was wanting to see that show!!
16606                     Har vondt i ryggen My back hurts
5223     Laying in bed with a book & some beautiful mus...
Name: text, Length: 4091, dtype: object> 



5. Vectorisation
 Le format de la matrice X_test_CV doit être 4091x15806 avec 44633 stored elements.

In [65]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_CV = vectorizer.fit_transform(X_train)
X_test_CV = vectorizer.transform(X_test)

# print(f"\n X_train_CV:\n{X_train_CV} \n")
print(f"\n X_test_CV:\n{X_test_CV.shape} \n")
# print(f"\n X_train_CV:\n{X_train_CV.toarray()} \n")
# print(f"\n X_test_CV:\n{X_test_CV.toarray()} \n")



 X_test_CV:
(4091, 15806) 



* 5 Entraîne maintenant une régression logistique avec les paramètres par défaut. Tu devrais obtenir les résultats suivants : 0.966 pour le test d'entraînement, et * 0.877 pour l'ensemble de test.
*  6 Étape bonus : essaye d'afficher 10 tweets qui ont été mal prédits (faux positifs ou faux négatifs). Aurais-tu fait mieux que l'algorithme ?

6. Entraînement d'une Régression Logistique

In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(max_iter=1000)
model.fit(X_train_CV, y_train)

# Évaluer le modèle
accuracy_train = model.score(X_train_CV, y_train)
accuracy_test = model.score(X_test_CV, y_test)

print(f"\n accuracy_train:{accuracy_train:.3f} ")
print(f"\n accuracy_test:{accuracy_test:.3f} ")



 accuracy_train:0.966 

 accuracy_test:0.877 


Étape Bonus : Affichage des Tweets Mal Prédits

In [70]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_CV)
# on selectionne toutes les instances de X_test  pour lesquelles les prédictions ne correspondent pas aux vraies valeurs. Cela vous donne un sous-ensemble de X_test contenant uniquement les instances mal prédites.
mal_predits = X_test[(y_pred != y_test)]

print(mal_predits.head(10))
# performance globale du modele avec f1
#   df.niveau1.value_counts()
#  faire un tfidf
# utiliser recall pour avoir le % de positifs bien predits
# print(classification_report(y, knn.predict(X)))

rapport_performance = classification_report(y_test, y_pred)
print(f"\n rapport_performance:\n{rapport_performance} \n")


2670     HollowbabesHere comes the utter shite #bgt <I ...
18731     SUFFICATION NO BREATHING. It`s okay. There`ll...
12054    i wanna vote for Miley Cyrus for the mtv movie...
21823    I love music so much that i`ve gone through pa...
18464    I can only message those who message me, if we...
2975     wish I could feel no pain (8)  but it`s ok, at...
3921                        so glad i`m not at uni anymore
5198      You`re not here. I hope you`re still resting....
467        you`re missing out, bb! i`m such a cereal nu...
15215     have an amazing time with your mommas tomorro...
Name: text, dtype: object

 rapport_performance:
              precision    recall  f1-score   support

    negative       0.86      0.88      0.87      1935
    positive       0.89      0.88      0.88      2156

    accuracy                           0.88      4091
   macro avg       0.88      0.88      0.88      4091
weighted avg       0.88      0.88      0.88      4091
 



# Quete 5 NLP 5 : TfIdf
Challenge - Tweetons !
Nous allons effectuer les mêmes missions que dans la quête précédente, afin de comparer les deux méthodes (TfIdf Vectorizer et CountVectorizer). Pour rappel, les missions étaient les suivantes :

* Importe cet ensemble de données de tweets dans un DataFrame.
* Ne garde que les tweets positifs et négatifs (tu excluras donc les neutral). Quel est le pourcentage de tweets positifs/négatifs ?
* Copie la colonne text dans une Série X, et la colonne sentiment dans une Série y. Applique un train test split avec le random_state = 32.
* Crée un modèle vectorizer avec scikit-learn en utilisant la méthode TfidfVectorizer. 
* Entraîne ton modèle sur X_train, puis crée une matrice de features X_train_CV. 
* Crée la matrice X_test_CV sans ré-entraîner le modèle. Le format de la matrice X_test_CV doit être 4091x15806 avec 44633 stored elements.
* Entraîne maintenant une régression logistique avec les paramètres par défaut. Tu devrais obtenir les résultats suivants : 0.932 pour le test d'entraînement, et 0.873 pour l'ensemble de test.


## Importe cet ensemble de données de tweets dans un DataFrame.


In [5]:
# Charger le DataFrame à partir du fichier CSV
df = pd.read_csv("tweets_train.csv")
print(f"\n df.info():\n{df.info()} \n")
# on ne garde pas les lignes \n\n
#  Filtre les tewets positifs et negatifs


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27480 entries, 0 to 27479
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27480 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27480 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB

 df.info():
None 



## Ne garde que les tweets positifs et négatifs (tu excluras donc les neutral). Quel est le pourcentage de tweets positifs/négatifs ?


In [6]:
df_filtered = df[df['sentiment'].isin(['positive', 'negative'])]
pourcentage_positif = (df_filtered['sentiment'] == 'positive').mean()
pourcentage_negatif = (df_filtered['sentiment'] == 'negative').mean()
print(f"\n pourcentage_positif:\n{pourcentage_positif:.2f} \n")
print(f"\n pourcentage_negatif:\n{pourcentage_negatif:.2f} \n")



 pourcentage_positif:
0.52 


 pourcentage_negatif:
0.48 



## Copie la colonne text dans une Série X, et la colonne sentiment dans une Série y. Applique un train test split avec le random_state = 32.


In [7]:
X = df_filtered['text']
y = df_filtered['sentiment']


# Creation train et test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=32)


## Crée un modèle vectorizer avec scikit-learn en utilisant la méthode TfidfVectorizer.

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Création du modèle TfidfVectorizer
model_TFIDF = TfidfVectorizer()



 X_test_CV:
(3273, 16428) 



 ## Entraîne ton modèle sur X_train, puis crée une matrice de features X_train_CV. 


In [25]:
# Entraînement du modèle sur X_train et création de la matrice de features X_train_CV
X_train_CV = model_TFIDF.fit_transform(X_train)

# Création de la matrice X_test_CV sans ré-entraîner le modèle
X_test_CV = model_TFIDF.transform(X_test)



 X_test_CV:
(3273, 16428) 



## Crée la matrice X_test_CV sans ré-entraîner le modèle. Le format de la matrice X_test_CV doit être 4091x15806 avec 44633 stored elements.


In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Création et entraînement du modèle de régression logistique
logistic_model = LogisticRegression()
logistic_model.fit(X_train_CV, y_train)

# Prédictions sur les ensembles d'entraînement et de test
y_train_pred = logistic_model.predict(X_train_CV)
y_test_pred = logistic_model.predict(X_test_CV)
# print(f"\n y_train_pred:\n{y_train_pred} \n")
# print(f"\n y_test_pred:\n{y_test_pred} \n")
print(f"\n X_test_CV:\n{X_test_CV.shape} \n")



 X_test_CV:
(3273, 16428) 



## Entraîne maintenant une régression logistique avec les paramètres par défaut. Tu devrais obtenir les résultats suivants : 0.932 pour le test d'entraînement, et 0.873 pour l'ensemble de test.

In [31]:
# Calcul de l'exactitude pour les deux ensembles
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"\n train_accuracy:{train_accuracy:.3f} ")
print(f"\n test_accuracy:{test_accuracy:.3f} ")



 train_accuracy:0.930 

 test_accuracy:0.872 


# NLP 6: Sentiment analysis


* Importation du dataset de tweets dans un DataFrame

Importez le dataset de tweets dans un DataFrame. Conservez uniquement les tweets positifs et négatifs (excluez les neutres). Calculez le pourcentage de tweets positifs/négatifs.

* Création de la fonction `clean`

- Créez une fonction que vous appellerez `clean`. Cette fonction prend en paramètre une phrase (un texte `str`) et retourne un texte (`str`) de tokens après application d'un stemmer ou d'un lemmatizer, séparés par des espaces.

- Vous pouvez tester votre fonction avec cette phrase, elle doit retourner quelque chose ressemblant à ce résultat :

``clean("You are better when I am well.")
"you be well when I be well ."``

* Suppression de la ponctuation et des stopwords
* Récupérez la liste des stopwords anglais depuis NLTK, et copiez-la dans une liste stopwordsenglish. Complétez votre fonction clean pour qu’elle supprime la ponctuation et les stopwords.

* Appliquez cette fonction clean à la colonne text de votre DataFrame. 

* Stockez le résultat dans une nouvelle colonne clean du DataFrame. (Le traitement peut durer 2 ou 3 minutes)
- Votre DataFrame doit maintenant ressembler à celui-ci (moins les pronoms, la ponctuation et peut-être d’autres mots en fonction des stopwords que vous avez nettoyés) :
 !DataFrame

* Préparation des données pour l’entraînement du modèle
- Copiez la colonne clean dans une Serie X, et la colonne sentiment dans une Serie y. 
- Appliquez un train-test split avec la taille du jeu d’entrainement à 0.75 avec le random_state = 32.

* Entraînement des modèles de classification
* Appliquez un CountVectorizer et entraînez des modèles de classification.

* Appliquez un TfidfVectorizer et entraînez des modèles de classification.
 
* Comparez les scores, quels paramètres permettent d’avoir les meilleurs scores ?
 
* Bonus : Amélioration du modèle. Maintenant, c’est à vous d’améliorer votre modèle :
* En cherchant des paramètres de modèles : par gridsearch et crossvalidation par exemple ;
* En changeant la préparation du texte : par exemple certaines ponctuations peuvent aider le modèle, le point d’exclamation notamment.

### Importation du dataset de tweets dans un DataFrame

- Importez le dataset de tweets dans un DataFrame. Conservez uniquement les tweets positifs et négatifs (excluez les neutres). Calculez le pourcentage de tweets positifs/négatifs.


In [2]:
# Charger le DataFrame à partir du fichier CSV
df = pd.read_csv("tweets_train.csv")
print(f"\n df.info():\n{df.info()} \n")
df_filtered = df[df['sentiment'].isin(['positive', 'negative'])]
print(f"\n df_filtered:\n{df_filtered.head(2)} \n")
pourcentage_positif = (df_filtered['sentiment'] == 'positive').mean()
pourcentage_negatif = (df_filtered['sentiment'] == 'negative').mean()
print(f"\n pourcentage_positif:\n{pourcentage_positif:.2f} \n")
print(f"\n pourcentage_negatif:\n{pourcentage_negatif:.2f} \n")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27480 entries, 0 to 27479
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27480 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27480 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB

 df.info():
None 


 df_filtered:
       textID                                            text selected_text  \
1  549e992a42   Sooo SAD I will miss you here in San Diego!!!      Sooo SAD   
2  088c60f138                       my boss is bullying me...   bullying me   

  sentiment  
1  negative  
2  negative   


 pourcentage_positif:
0.52 


 pourcentage_negatif:
0.48 



### Création de la fonction `clean`

- Créez une fonction que vous appellerez `clean`. Cette fonction prend en paramètre une phrase (un texte `str`) et retourne un texte (`str`) de tokens après application d'un stemmer ou d'un lemmatizer, séparés par des espaces.

- Vous pouvez tester votre fonction avec cette phrase, elle doit retourner quelque chose ressemblant à ce résultat :

``clean("You are better when I am well.")
"you be well when I be well ."``

### Suppression de la ponctuation et des stopwords
### Récupérez la liste des stopwords anglais depuis NLTK, et copiez-la dans une liste stopwordsenglish. Complétez votre fonction clean pour qu’elle supprime la ponctuation et les stopwords.

In [3]:
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from textblob import TextBlob, Word
import nltk
import string
import re

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
# Cette fonction prepare le texte en stemmatisation et lemmatization avec 3 modeles différents. On lui passe aussi une liste de caractéres de ponctuation que l'on désire garder car en effet un ! ou un  ? on un sens dans un tweet
def clean(text,keep_ponctuation):
    # Initialisation des modèles de stemming et lemmatization
    stemmer_snowball = SnowballStemmer('english')
    lemmatizer_wordnet = WordNetLemmatizer()
    lemmatizer_TextBlob = TextBlob(text)

    # Si le texte est vide, retournez des listes vides sinon on ne pourra pas créer les colonnes pd.Series([stemmer_snowball_tokens, lemmatizer_wordnet_tokens, lemmatized_TextBlob_tokens])
    # if not  isinstance(text, str) or not text:
    #     return [], [], []

    # print(f" stemmer_snowball:{stemmer_snowball} ")
    # print(f" lemmatizer_worldnet:{lemmatizer_worldnet} ")
    # print(f" lemmatizer_TextBlob :{lemmatizer_TextBlob} ")


    # print(f" text:{text} ")

    # Chargement des stopwords anglais
    stopwords_english = set(stopwords.words('english'))

    # Tokenisation
    tokens = nltk.word_tokenize(text.lower())

    # # Suppression des caractères spéciaux
    # text = re.sub(r'[^a-zA-Z0-9]', '', text)

    # Caractères de ponctuation à conserver
    keep_ponctuation
    # Création une chaîne de caractères de ponctuation personnalisée
    if  keep_ponctuation:
        custom_punctuation = "".join(ch for ch in string.punctuation if ch not in keep_ponctuation)
    else:
        custom_punctuation=string.punctuation
    # print(f" custom_punctuation:{custom_punctuation} ")

    # Stemming avec SnowballStemmer
    stemmer_snowball_tokens = [stemmer_snowball.stem(
        token) for token in tokens if token not in stopwords_english and token not in custom_punctuation]

    # Lemmatization with WordNetLemmatizer
    lemmatizer_worldnet_tokens = [lemmatizer_wordnet.lemmatize(token) for token in tokens if token not in stopwords_english and token not in custom_punctuation]

    # Lemmatisation avec TextBlob
    lemmatized_TextBlob_tokens = [Word(word).lemmatize().lower() for word in lemmatizer_TextBlob.words if word.lower(
    ) not in stopwords_english and word not in custom_punctuation]

    # print(f"\n stemmer_snowball_tokens:{stemmer_snowball_tokens} ")
    # print(f"lemmatizer_worldnet_tokens :{lemmatizer_worldnet_tokens} ")
    # print(f" lemmatized_TextBlob_tokens:{lemmatized_TextBlob_tokens} ")

    return stemmer_snowball_tokens, lemmatizer_worldnet_tokens, lemmatized_TextBlob_tokens


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\romar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\romar\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\romar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\romar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Appliquez cette fonction clean à la colonne text de votre DataFrame. 
- Stockez le résultat dans une nouvelle colonne clean du DataFrame. (Le traitement peut durer 2 ou 3 minutes)

In [4]:
# Liste des caractères de ponctuation à conserver
keep_chars = "! ? , ."
#  copie du DataFrame pour éviter le SettingWithCopyWarning
df_filtered_cleaned = df_filtered.copy()

# Appliquer la fonction clean et séparer les résultats
results = df_filtered_cleaned['text'].apply(lambda x: clean(x, keep_chars))

# Créer de nouvelles colonnes pour chaque résultat dans la copie du DataFrame
df_filtered_cleaned['clean_snowball'] = results.apply(lambda x: x[0])
df_filtered_cleaned['clean_wordnet'] = results.apply(lambda x: x[1])
df_filtered_cleaned['clean_textblob'] = results.apply(lambda x: x[2])

# Détokeniser chaque colonne pour avoir des chaînes de caractères et non des listes
df_filtered_cleaned['clean_snowball'] = df_filtered_cleaned['clean_snowball'].apply(lambda tokens: ' '.join(tokens))
df_filtered_cleaned['clean_wordnet'] = df_filtered_cleaned['clean_wordnet'].apply(lambda tokens: ' '.join(tokens))
df_filtered_cleaned['clean_textblob'] = df_filtered_cleaned['clean_textblob'].apply(lambda tokens: ' '.join(tokens))

# Afficher les premières lignes pour vérifier
# print(df_filtered_copy[['clean_snowball', 'clean_wordnet', 'clean_textblob']].head())
df_filtered_cleaned


,textID,text,selected_text,sentiment,clean_snowball,clean_wordnet,clean_textblob
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sooo sad miss san diego ! ! !,sooo sad miss san diego ! ! !,sooo sad miss san diego
2,088c60f138,my boss is bullying me...,bullying me,negative,boss bulli ...,bos bullying ...,bos bullying
3,9642c003ef,what interview! leave me alone,leave me alone,negative,interview ! leav alon,interview ! leave alone,interview leave alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"son , put releas alreadi bought","son , put release already bought",sons put release already bought
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive,2am feed babi fun smile coo,2am feeding baby fun smile coo,2am feeding baby fun smile coo
...,...,...,...,...,...,...,...
27474,b78ec00df5,enjoy ur night,enjoy,positive,enjoy ur night,enjoy ur night,enjoy ur night
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,wish could come see u denver husband lost job ...,wish could come see u denver husband lost job ...,wish could come see u denver husband lost job ...
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,"wonder rake . client made clear .net , forc de...","wondered rake . client made clear .net , force...",wondered rake client made clear net force devs...
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,yay good . enjoy break probabl need hectic wee...,yay good . enjoy break probably need hectic we...,yay good enjoy break probably need hectic week...


In [ ]:
# #  copie du DataFrame pour éviter le SettingWithCopyWarning
# df_filtered_copy = df_filtered.copy()

# # Appliquer la fonction clean et séparer les résultats
# results = df_filtered_copy['text'].apply(clean)

# # Créer de nouvelles colonnes pour chaque résultat dans la copie du DataFrame
# df_filtered_copy['clean_snowball'] = results.apply(lambda x: x[0])
# df_filtered_copy['clean_wordnet'] = results.apply(lambda x: x[1])
# df_filtered_copy['clean_textblob'] = results.apply(lambda x: x[2])

# # Déokéniser chaque colonne de facon à avoir des chaines de caractéres et non des listes
# df_filtered_copy['clean_snowball'] = df_filtered_copy['clean_snowball'].apply(lambda tokens: ' '.join(tokens))
# df_filtered_copy['clean_wordnet'] = df_filtered_copy['clean_wordnet'].apply(lambda tokens: ' '.join(tokens))
# df_filtered_copy['clean_textblob'] = df_filtered_copy['clean_textblob'].apply(lambda tokens: ' '.join(tokens))

# # Afficher les premières lignes pour vérifier
# # print(df_filtered_copy[['clean_snowball', 'clean_wordnet', 'clean_textblob']].head())

# # Maintenant, df_filtered_copy contient les nouvelles colonnes sans warnings
# df_filtered_copy



### Préparation des données pour l’entraînement du modèle
- Copiez la colonne clean dans une Serie X, et la colonne sentiment dans une Serie y. 
- Appliquez un train-test split avec la taille du jeu d’entrainement à 0.75 avec le random_state = 32.

In [34]:
# Copie de la colonne clean dans une Serie X, et de la colonne sentiment dans une Serie y
X = df_filtered_cleaned['clean_wordnet']
y = df_filtered_cleaned['sentiment']
X


1                            sooo sad miss san diego ! ! !
2                                         bos bullying ...
3                                  interview ! leave alone
4                         son , put release already bought
6                           2am feeding baby fun smile coo
                               ...                        
27474                                       enjoy ur night
27475    wish could come see u denver husband lost job ...
27476    wondered rake . client made clear .net , force...
27477    yay good . enjoy break probably need hectic we...
27478                                              worth .
Name: clean_wordnet, Length: 16363, dtype: object

In [35]:
# Application d'un train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=32)


In [224]:
X


1                            sooo sad miss san diego ! ! !
2                                           boss bulli ...
3                                    interview ! leav alon
4                          son , put releas alreadi bought
6                              2am feed babi fun smile coo
                               ...                        
27474                                       enjoy ur night
27475    wish could come see u denver husband lost job ...
27476    wonder rake . client made clear .net , forc de...
27477    yay good . enjoy break probabl need hectic wee...
27478                                              worth .
Name: clean_snowball, Length: 16363, dtype: object

### Entraînement des modèles de classification
### Appliquez un CountVectorizer et entraînez des modèles de classification.

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
# Conversion du texte en matrice creuse pour avoir un bag_of_word
model_vectorizer = CountVectorizer()
X_train_CV = model_vectorizer.fit_transform(X_train)
X_test_CV = model_vectorizer.transform(X_test)

# print(f"\n X_train_CV:\n{X_train_CV} \n")
print(f"\n X_test_CV:\n{X_test_CV.shape} \n")



 X_test_CV:
(4091, 14657) 



### Appliquez un TfidfVectorizer et entraînez des modèles de classification.

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Création du modèle TfidfVectorizer
model_TFIDF = TfidfVectorizer(min_df=1)

# Entraînement du modèle sur X_train et création de la matrice de features X_train_CV
X_train_tfidf= model_TFIDF.fit_transform(X_train)

# Création de la matrice X_test_CV sans ré-entraîner le modèle
X_test_tfidf= model_TFIDF.transform(X_test)
print(f"\n X_test_CV:\n{X_test_tfidf.shape} \n")





 X_test_CV:
(4091, 14657) 



## Entrainnement des modéles avec la logisticRegression

In [38]:
from sklearn.linear_model import LogisticRegression

#  Entrainnement sur CountVectorizer
model_vectorizer = LogisticRegression(max_iter=10000)
model_vectorizer.fit(X_train_CV, y_train)

#  Entrainnement sur TFIDF
model_TFIDF = LogisticRegression(max_iter=10000)
model_TFIDF.fit(X_train_tfidf, y_train)


LogisticRegression(max_iter=10000)

### Comparez les scores, quels paramètres permettent d’avoir les meilleurs scores ?

In [39]:
from sklearn.metrics import accuracy_score
# Évaluer le modèle Bag of word avec CountVectorizer
accuracy_train = model_vectorizer.score(X_train_CV, y_train)
accuracy_test = model_vectorizer.score(X_test_CV, y_test)
print(f" accuracy_train CountVectorizer :{accuracy_train*100:.2f}% accuracy_test: {accuracy_test*100:.2f}% ")
# Évaluer le modèle TFIDF
accuracy_train = model_TFIDF.score(X_train_tfidf, y_train)
accuracy_test = model_TFIDF.score(X_test_tfidf, y_test)
print(f" accuracy_train TFIDF:{accuracy_train*100:.2f}% accuracy_test:{accuracy_test*100:.2f}% ")
# sans min_df
# accuracy_train CountVectorizer :95.44% accuracy_test: 86.53%
#  accuracy_train TFIDF:92.666% accuracy_test:86.923%
#  avec min_df=1 sur la colonne texte
#  accuracy_train CountVectorizer :96.63% accuracy_test: 87.73%
#  accuracy_train TFIDF:93.21% accuracy_test:87.31%
#  avec min_df=1 sur la colonne wordnet
#  accuracy_train CountVectorizer :95.95% accuracy_test: 86.60%
#  accuracy_train TFIDF:93.08% accuracy_test:86.78%



 accuracy_train CountVectorizer :95.95% accuracy_test: 86.60% 
 accuracy_train TFIDF:93.08% accuracy_test:86.78% 


### Bonus : Amélioration du modèle. Maintenant, c’est à vous d’améliorer votre modèle :
### En cherchant des paramètres de modèles : par gridsearch et crossvalidation par exemple ;
### En changeant la préparation du texte : par exemple certaines ponctuations peuvent aider le modèle, le point d’exclamation notamment.

en supprimmant toute la poonctuatin les scoares étaient de 
 accuracy_train CountVectorizer :0.95 accuracy_test: 0.87 
 accuracy_train TFIDF:0.93 accuracy_test:0.87 

 en laissant les ? ! . , le score est de :
accuracy_train CountVectorizer :95.4% accuracy_test: 86.5% 
accuracy_train TFIDF:92.66% accuracy_test:86.92% 


In [234]:
df_filtered_cleaned


,textID,text,selected_text,sentiment,clean_snowball,clean_wordnet,clean_textblob
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sooo sad miss san diego ! ! !,sooo sad miss san diego ! ! !,sooo sad miss san diego
2,088c60f138,my boss is bullying me...,bullying me,negative,boss bulli ...,bos bullying ...,bos bullying
3,9642c003ef,what interview! leave me alone,leave me alone,negative,interview ! leav alon,interview ! leave alone,interview leave alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"son , put releas alreadi bought","son , put release already bought",sons put release already bought
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive,2am feed babi fun smile coo,2am feeding baby fun smile coo,2am feeding baby fun smile coo
...,...,...,...,...,...,...,...
27474,b78ec00df5,enjoy ur night,enjoy,positive,enjoy ur night,enjoy ur night,enjoy ur night
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,wish could come see u denver husband lost job ...,wish could come see u denver husband lost job ...,wish could come see u denver husband lost job ...
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,"wonder rake . client made clear .net , forc de...","wondered rake . client made clear .net , force...",wondered rake client made clear net force devs...
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,yay good . enjoy break probabl need hectic wee...,yay good . enjoy break probably need hectic we...,yay good enjoy break probably need hectic week...


In [240]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression

# Liste des colonnes à tester
columns = ['clean_snowball', 'clean_wordnet', 'clean_textblob']

# Définir les paramètres à tester
parameters = {
    'clf__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'clf__penalty': ['l1', 'l2'],
    'clf__max_iter': [100, 200, 500, 1000],
    'clf__solver': ['liblinear']  # Ajout du paramètre solver
}

# Boucle sur chaque colonne
for column in columns:
    # Sélectionner la colonne
    X = df_filtered_cleaned[column]
    # Liste des caractères à supprimer
    drop_chars = ",."

    # Boucle sur chaque caractère à supprimer
    for char in drop_chars:
        X = X.str.replace(char, "")
    # print(f"\n X:{X} ")
    y = df_filtered['sentiment']  # Remplacez par votre colonne cible

    # Diviser les données en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=32)

    # Définir le pipeline pour chaque type de vectoriseur
    for vectorizer in [CountVectorizer(), TfidfVectorizer()]:
        pipeline = Pipeline([
            ('features', vectorizer),
            ('clf', LogisticRegression())
        ])

        # Initialiser la recherche sur grille
        grid_search = GridSearchCV(pipeline, parameters, cv=5,scoring='accuracy', n_jobs=-1)

        # Ajuster le modèle
        grid_search.fit(X_train, y_train)

        # Afficher les meilleurs paramètres
        print(f"Best parameters for {column} with {vectorizer.__class__.__name__}: {grid_search.best_params_}")


Best parameters for clean_snowball with CountVectorizer: {'clf__C': 1, 'clf__max_iter': 100, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}
Best parameters for clean_snowball with TfidfVectorizer: {'clf__C': 1, 'clf__max_iter': 100, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}
Best parameters for clean_wordnet with CountVectorizer: {'clf__C': 1, 'clf__max_iter': 100, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}


c:\Users\romar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\romar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best parameters for clean_wordnet with TfidfVectorizer: {'clf__C': 1, 'clf__max_iter': 100, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}
Best parameters for clean_textblob with CountVectorizer: {'clf__C': 1, 'clf__max_iter': 100, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}


c:\Users\romar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\romar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best parameters for clean_textblob with TfidfVectorizer: {'clf__C': 1, 'clf__max_iter': 100, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}


In [242]:
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.exceptions import NotFittedError

# Liste des colonnes à tester
columns = ['clean_snowball', 'clean_wordnet', 'clean_textblob']

# Définir les paramètres à tester
parameters = {
    'clf__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'clf__penalty': ['l1', 'l2'],
    'clf__max_iter': [500, 1000],
    'clf__solver': ['liblinear']  # Ajout du paramètre solver
}

# Boucle sur chaque colonne
for column in columns:
    # Sélectionner la colonne
    X = df_filtered_cleaned[column]
    # Liste des caractères à supprimer
    drop_chars = ",."

    # Boucle sur chaque caractère à supprimer
    for char in drop_chars:
        X = X.str.replace(char, "")
    # print(f"\n X:{X} ")
    y = df_filtered['sentiment']  # Remplacez par votre colonne cible

    # Diviser les données en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=32)

    # Définir le pipeline pour chaque type de vectoriseur
    for vectorizer in [CountVectorizer(), TfidfVectorizer()]:
        pipeline = Pipeline([
            ('features', vectorizer),
            ('clf', LogisticRegression())
        ])

        # Initialiser la recherche sur grille
        grid_search = GridSearchCV(pipeline, parameters, cv=5,scoring='accuracy', n_jobs=-1)

        # Ajuster le modèle
        try:
            grid_search.fit(X_train, y_train)
        except ValueError as e:
            print(f"Erreur lors de l'ajustement du modèle : {e}")
            continue
        except NotFittedError as e:
            print(f"Erreur lors de l'ajustement du modèle : {e}")
            continue

        # Prédire les valeurs de test
        y_pred = grid_search.predict(X_test)

        # Calculer le score
        score = grid_search.score(X_test, y_test)

        # Afficher le score et le rapport de classification
        print(f"\nScore for {column} with {vectorizer.__class__.__name__}: {score}")
        print(classification_report(y_test, y_pred))

        # Afficher les meilleurs paramètres
        print(f"Best parameters for {column} with {vectorizer.__class__.__name__}: {grid_search.best_params_}")



Score for clean_snowball with CountVectorizer: 0.8650696651185529
              precision    recall  f1-score   support

    negative       0.85      0.86      0.86      1935
    positive       0.88      0.87      0.87      2156

    accuracy                           0.87      4091
   macro avg       0.86      0.86      0.86      4091
weighted avg       0.87      0.87      0.87      4091

Best parameters for clean_snowball with CountVectorizer: {'clf__C': 1, 'clf__max_iter': 500, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}

Score for clean_snowball with TfidfVectorizer: 0.8684918112930824
              precision    recall  f1-score   support

    negative       0.86      0.87      0.86      1935
    positive       0.88      0.87      0.87      2156

    accuracy                           0.87      4091
   macro avg       0.87      0.87      0.87      4091
weighted avg       0.87      0.87      0.87      4091

Best parameters for clean_snowball with TfidfVectorizer: {'clf__C': 1

c:\Users\romar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\romar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(



Score for clean_wordnet with TfidfVectorizer: 0.8684918112930824
              precision    recall  f1-score   support

    negative       0.85      0.87      0.86      1935
    positive       0.88      0.87      0.87      2156

    accuracy                           0.87      4091
   macro avg       0.87      0.87      0.87      4091
weighted avg       0.87      0.87      0.87      4091

Best parameters for clean_wordnet with TfidfVectorizer: {'clf__C': 1, 'clf__max_iter': 500, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}

Score for clean_textblob with CountVectorizer: 0.8645807870936202
              precision    recall  f1-score   support

    negative       0.85      0.87      0.86      1935
    positive       0.88      0.86      0.87      2156

    accuracy                           0.86      4091
   macro avg       0.86      0.86      0.86      4091
weighted avg       0.86      0.86      0.86      4091

Best parameters for clean_textblob with CountVectorizer: {'clf__C': 1, 

c:\Users\romar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\romar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(



Score for clean_textblob with TfidfVectorizer: 0.8680029332681496
              precision    recall  f1-score   support

    negative       0.85      0.87      0.86      1935
    positive       0.88      0.87      0.87      2156

    accuracy                           0.87      4091
   macro avg       0.87      0.87      0.87      4091
weighted avg       0.87      0.87      0.87      4091

Best parameters for clean_textblob with TfidfVectorizer: {'clf__C': 1, 'clf__max_iter': 500, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.exceptions import NotFittedError

# Liste des colonnes à tester
columns = ['clean_snowball', 'clean_wordnet', 'clean_textblob']

# Définir les paramètres à tester pour chaque modèle
parameters = {
    'LogisticRegression': {
        'clf__C': [0.001, 0.01, 0.1, 1, 10, 100],
        'clf__penalty': ['l2'],
        'clf__max_iter': [ 1000,2000],
        'clf__solver': ['lbfgs']
    },
    'RandomForestClassifier': {
        'clf__n_estimators': [100, 200, 500],
        'clf__max_depth': [None, 10, 20, 30],
        'clf__min_samples_split': [2, 5, 10]
    },
    'SVC': {
        'clf__C': [0.1, 1, 10, 100],
        'clf__gamma': ['scale', 'auto'],
        'clf__kernel': ['linear', 'rbf']
    }
}

# Boucle sur chaque colonne
for column in columns:
    # Sélectionner la colonne
    X = df_filtered_cleaned[column]
    # Liste des caractères à supprimer
    drop_chars = ",."

    # Boucle sur chaque caractère à supprimer
    for char in drop_chars:
        X = X.str.replace(char, "")
    # print(f"\n X:{X} ")
    y = df_filtered['sentiment']  # Remplacez par votre colonne cible

    # Diviser les données en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=32)

    # Définir le pipeline pour chaque type de vectoriseur
    for vectorizer in [CountVectorizer(), TfidfVectorizer()]:
        # Boucle sur chaque modèle
        for model_name, model in [('LogisticRegression', LogisticRegression()), ('RandomForestClassifier', RandomForestClassifier()), ('SVC', SVC())]:
            pipeline = Pipeline([
                ('features', vectorizer),
                ('clf', model)
            ])

            # Initialiser la recherche sur grille
            grid_search = GridSearchCV(pipeline, parameters[model_name], cv=5, scoring='accuracy', n_jobs=-1)

            # Ajuster le modèle
            try:
                grid_search.fit(X_train, y_train)
            except ValueError as e:
                print(f"Erreur lors de l'ajustement du modèle {model_name} : {e}")
                continue
            except NotFittedError as e:
                print(f"Erreur lors de l'ajustement du modèle {model_name} : {e}")
                continue

            # Prédire les valeurs de test
            y_pred = grid_search.predict(X_test)

            # Calculer le score
            score = grid_search.score(X_test, y_test)

            # Afficher le score et le rapport de classification
            print(f"\nScore for {column} with {vectorizer.__class__.__name__} and {model_name}: {score}")
            print(classification_report(y_test, y_pred))

            # Afficher les meilleurs paramètres
            print(f"Best parameters for {column} with {vectorizer.__class__.__name__} and {model_name}: {grid_search.best_params_}")


version sans standardisation ni ACP

In [5]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.exceptions import NotFittedError
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Liste des colonnes à tester
columns = ['clean_snowball', 'clean_wordnet', 'clean_textblob']
columns = ['text']

# Définir les paramètres à tester pour chaque modèle
parameters = {
    'LogisticRegression': {
        'clf__C': [0.001, 0.01, 0.1, 1, 10, 100],
        'clf__penalty': ['l2'],
        'clf__max_iter': [5000,10000],
        'clf__solver': ['lbfgs']
    },
    'RandomForestClassifier': {
        'clf__n_estimators': [100, 200, 500],
        'clf__max_depth': [None, 10, 20, 30],
        'clf__min_samples_split': [2, 5, 10]
    },
    'SVC': {
        'clf__C': [0.1, 1, 10, 100],
        'clf__gamma': ['scale', 'auto'],
        'clf__kernel': ['linear', 'rbf']
    }
}

# Boucle sur chaque colonne
for column in columns:
    # Sélectionner la colonne
    X = df_filtered_cleaned[column]
    # Liste des caractères à supprimer
    drop_chars = ",."

    # Boucle sur chaque caractère à supprimer
    for char in drop_chars:
        X = X.str.replace(char, "")
    # print(f"\n X:{X} ")
    y = df_filtered['sentiment']  # Remplacez par votre colonne cible

    # Diviser les données en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=32)

    # Définir le pipeline pour chaque type de vectoriseur
    for vectorizer in [CountVectorizer(), TfidfVectorizer()]:
        # Boucle sur chaque modèle
        for model_name, model in [('LogisticRegression', LogisticRegression()),('RandomForestClassifier', RandomForestClassifier()),('SVC', SVC())]:
            pipeline = Pipeline([
                ('features', vectorizer),
                ('clf', model)
            ])

            # Initialiser la recherche sur gridsearch
            grid_search = GridSearchCV(pipeline, parameters[model_name], cv=5, scoring='accuracy', n_jobs=-1)

            # Ajuster le modèle
            try:
                grid_search.fit(X_train, y_train)
            except ValueError as e:
                print(f"Erreur lors de l'ajustement du modèle {model_name} : {e}")
                continue
            except NotFittedError as e:
                print(f"Erreur lors de l'ajustement du modèle {model_name} : {e}")
                continue

            # Prédire les valeurs de test
            y_pred = grid_search.predict(X_test)

            # Calculer le score
            score = grid_search.score(X_test, y_test)

            # Afficher le score et le rapport de classification
            print(f"\nScore for {column} with suppression of {drop_chars} and {vectorizer.__class__.__name__} and {model_name}: {score*100:.2f}")
            print(classification_report(y_test, y_pred))

            # Afficher les meilleurs paramètres
            print(f"Best parameters for {column} with {vectorizer.__class__.__name__} and {model_name}: {grid_search.best_params_}")



Score for text with suppression of ,. and CountVectorizer and LogisticRegression: 87.24
              precision    recall  f1-score   support

    negative       0.86      0.87      0.87      1935
    positive       0.89      0.87      0.88      2156

    accuracy                           0.87      4091
   macro avg       0.87      0.87      0.87      4091
weighted avg       0.87      0.87      0.87      4091

Best parameters for text with CountVectorizer and LogisticRegression: {'clf__C': 1, 'clf__max_iter': 5000, 'clf__penalty': 'l2', 'clf__solver': 'lbfgs'}

Score for text with suppression of ,. and CountVectorizer and RandomForestClassifier: 86.19
              precision    recall  f1-score   support

    negative       0.84      0.87      0.86      1935
    positive       0.88      0.85      0.87      2156

    accuracy                           0.86      4091
   macro avg       0.86      0.86      0.86      4091
weighted avg       0.86      0.86      0.86      4091

Best parame


Score for clean_snowball with CountVectorizer and LogisticRegression: 0.8655585431434857
              precision    recall  f1-score   support

    negative       0.85      0.86      0.86      1935
    positive       0.88      0.87      0.87      2156

    accuracy                           0.87      4091
   macro avg       0.87      0.87      0.87      4091
weighted avg       0.87      0.87      0.87      4091

Best parameters for clean_snowball with CountVectorizer and LogisticRegression: {'clf__C': 1, 'clf__max_iter': 5000, 'clf__penalty': 'l2', 'clf__solver': 'lbfgs'}

Score for clean_snowball with CountVectorizer and RandomForestClassifier: 0.8628697140063554
              precision    recall  f1-score   support

    negative       0.85      0.86      0.86      1935
    positive       0.87      0.87      0.87      2156

    accuracy                           0.86      4091
   macro avg       0.86      0.86      0.86      4091
weighted avg       0.86      0.86      0.86      4091

Best parameters for clean_snowball with CountVectorizer and RandomForestClassifier: {'clf__max_depth': None, 'clf__min_samples_split': 10, 'clf__n_estimators': 500}

Score for clean_snowball with CountVectorizer and SVC: 0.8645807870936202
              precision    recall  f1-score   support

    negative       0.84      0.88      0.86      1935
    positive       0.89      0.85      0.87      2156

    accuracy                           0.86      4091
   macro avg       0.86      0.87      0.86      4091
weighted avg       0.87      0.86      0.86      4091

Best parameters for clean_snowball with CountVectorizer and SVC: {'clf__C': 0.1, 'clf__gamma': 'scale', 'clf__kernel': 'linear'}

Score for clean_snowball with TfidfVectorizer and LogisticRegression: 0.8684918112930824
              precision    recall  f1-score   support

    negative       0.86      0.87      0.86      1935
    positive       0.88      0.87      0.87      2156

    accuracy                           0.87      4091
   macro avg       0.87      0.87      0.87      4091
weighted avg       0.87      0.87      0.87      4091

Best parameters for clean_snowball with TfidfVectorizer and LogisticRegression: {'clf__C': 1, 'clf__max_iter': 5000, 'clf__penalty': 'l2', 'clf__solver': 'lbfgs'}

Score for clean_snowball with TfidfVectorizer and RandomForestClassifier: 0.8562698606697629
              precision    recall  f1-score   support

    negative       0.85      0.85      0.85      1935
    positive       0.86      0.86      0.86      2156

    accuracy                           0.86      4091
   macro avg       0.86      0.86      0.86      4091
weighted avg       0.86      0.86      0.86      4091

Best parameters for clean_snowball with TfidfVectorizer and RandomForestClassifier: {'clf__max_depth': None, 'clf__min_samples_split': 5, 'clf__n_estimators': 500}

Score for clean_snowball with TfidfVectorizer and SVC: 0.8724028354925446
              precision    recall  f1-score   support

    negative       0.86      0.87      0.87      1935
    positive       0.88      0.87      0.88      2156

    accuracy                           0.87      4091
   macro avg       0.87      0.87      0.87      4091
weighted avg       0.87      0.87      0.87      4091

Best parameters for clean_snowball with TfidfVectorizer and SVC: {'clf__C': 1, 'clf__gamma': 'scale', 'clf__kernel': 'rbf'}

Score for clean_wordnet with CountVectorizer and LogisticRegression: 0.8662918601808849
              precision    recall  f1-score   support

    negative       0.85      0.87      0.86      1935
    positive       0.88      0.86      0.87      2156

    accuracy                           0.87      4091
   macro avg       0.87      0.87      0.87      4091
weighted avg       0.87      0.87      0.87      4091

Best parameters for clean_wordnet with CountVectorizer and LogisticRegression: {'clf__C': 1, 'clf__max_iter': 5000, 'clf__penalty': 'l2', 'clf__solver': 'lbfgs'}

Score for clean_wordnet with CountVectorizer and RandomForestClassifier: 0.8596920068442924
              precision    recall  f1-score   support

    negative       0.85      0.86      0.85      1935
    positive       0.87      0.86      0.87      2156

    accuracy                           0.86      4091
   macro avg       0.86      0.86      0.86      4091
weighted avg       0.86      0.86      0.86      4091

Best parameters for clean_wordnet with CountVectorizer and RandomForestClassifier: {'clf__max_depth': None, 'clf__min_samples_split': 10, 'clf__n_estimators': 500}

Score for clean_wordnet with CountVectorizer and SVC: 0.8614030799315571
              precision    recall  f1-score   support

    negative       0.84      0.87      0.86      1935
    positive       0.88      0.85      0.87      2156

    accuracy                           0.86      4091
   macro avg       0.86      0.86      0.86      4091
weighted avg       0.86      0.86      0.86      4091

Best parameters for clean_wordnet with CountVectorizer and SVC: {'clf__C': 1, 'clf__gamma': 'scale', 'clf__kernel': 'rbf'}

Score for clean_wordnet with TfidfVectorizer and LogisticRegression: 0.8684918112930824
              precision    recall  f1-score   support

    negative       0.85      0.87      0.86      1935
    positive       0.88      0.87      0.87      2156

    accuracy                           0.87      4091
   macro avg       0.87      0.87      0.87      4091
weighted avg       0.87      0.87      0.87      4091

Best parameters for clean_wordnet with TfidfVectorizer and LogisticRegression: {'clf__C': 1, 'clf__max_iter': 5000, 'clf__penalty': 'l2', 'clf__solver': 'lbfgs'}

Score for clean_wordnet with TfidfVectorizer and RandomForestClassifier: 0.8543143485700317
              precision    recall  f1-score   support

    negative       0.84      0.85      0.85      1935
    positive       0.86      0.86      0.86      2156

    accuracy                           0.85      4091
   macro avg       0.85      0.85      0.85      4091
weighted avg       0.85      0.85      0.85      4091

Best parameters for clean_wordnet with TfidfVectorizer and RandomForestClassifier: {'clf__max_depth': None, 'clf__min_samples_split': 5, 'clf__n_estimators': 200}

Score for clean_wordnet with TfidfVectorizer and SVC: 0.8694695673429479
              precision    recall  f1-score   support

    negative       0.86      0.87      0.86      1935
    positive       0.88      0.87      0.88      2156

    accuracy                           0.87      4091
   macro avg       0.87      0.87      0.87      4091
weighted avg       0.87      0.87      0.87      4091

Best parameters for clean_wordnet with TfidfVectorizer and SVC: {'clf__C': 1, 'clf__gamma': 'scale', 'clf__kernel': 'rbf'}

Score for clean_textblob with CountVectorizer and LogisticRegression: 0.8643363480811538
              precision    recall  f1-score   support

    negative       0.85      0.87      0.86      1935
    positive       0.88      0.86      0.87      2156

    accuracy                           0.86      4091
   macro avg       0.86      0.86      0.86      4091
weighted avg       0.86      0.86      0.86      4091

Best parameters for clean_textblob with CountVectorizer and LogisticRegression: {'clf__C': 1, 'clf__max_iter': 5000, 'clf__penalty': 'l2', 'clf__solver': 'lbfgs'}

Score for clean_textblob with CountVectorizer and RandomForestClassifier: 0.8579809337570277
              precision    recall  f1-score   support

    negative       0.85      0.85      0.85      1935
    positive       0.87      0.86      0.87      2156

    accuracy                           0.86      4091
   macro avg       0.86      0.86      0.86      4091
weighted avg       0.86      0.86      0.86      4091

Best parameters for clean_textblob with CountVectorizer and RandomForestClassifier: {'clf__max_depth': None, 'clf__min_samples_split': 5, 'clf__n_estimators': 500}


version avec ACP sans standadisation

In [246]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.exceptions import NotFittedError
from sklearn.decomposition import PCA
from sklearn.pipeline import FeatureUnion

# Liste des colonnes à tester
columns = ['clean_snowball', 'clean_wordnet', 'clean_textblob']

# Définir les paramètres à tester pour chaque modèle
parameters = {
    'LogisticRegression': {
        'clf__C': [0.001, 0.01, 0.1, 1, 10, 100],
        'clf__penalty': ['l2'],
        'clf__max_iter': [100, 200, 500, 1000],
        'clf__solver': ['lbfgs']
    },
    'RandomForestClassifier': {
        'clf__n_estimators': [100, 200, 500],
        'clf__max_depth': [None, 10, 20, 30],
        'clf__min_samples_split': [2, 5, 10]
    },
    'SVC': {
        'clf__C': [0.1, 1, 10, 100],
        'clf__gamma': ['scale', 'auto'],
        'clf__kernel': ['linear', 'rbf']
    }
}

# Boucle sur chaque colonne
for column in columns:
    # Sélectionner la colonne
    X = df_filtered_cleaned[column]
    # Liste des caractères à supprimer
    drop_chars = ",."

    # Boucle sur chaque caractère à supprimer
    for char in drop_chars:
        X = X.str.replace(char, "")
    # print(f"\n X:{X} ")
    y = df_filtered['sentiment']  # Remplacez par votre colonne cible

    # Diviser les données en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=32)

    # Définir le pipeline pour chaque type de vectoriseur
    for vectorizer in [CountVectorizer(), TfidfVectorizer()]:
        # Boucle sur chaque modèle
        for model_name, model in [('LogisticRegression', LogisticRegression()), ('RandomForestClassifier', RandomForestClassifier()), ('SVC', SVC())]:
            pipeline = Pipeline([
                ('features', FeatureUnion([
                    ('vectorizer', vectorizer),
                    ('pca', PCA(n_components=2))
                ])),
                ('clf', model)
            ])

            # Initialiser la recherche sur grille
            grid_search = GridSearchCV(pipeline, parameters[model_name], cv=5)

            # Ajuster le modèle
            try:
                grid_search.fit(X_train, y_train)
            except ValueError as e:
                print(f"Erreur lors de l'ajustement du modèle {model_name} : {e}")
                continue
            except NotFittedError as e:
                print(f"Erreur lors de l'ajustement du modèle {model_name} : {e}")
                continue

            # Prédire les valeurs de test
            y_pred = grid_search.predict(X_test)

            # Calculer le score
            score = grid_search.score(X_test, y_test)

            # Afficher le score et le rapport de classification
            print(f"\nScore for {column} with {vectorizer.__class__.__name__} and {model_name}: {score}")
            print(classification_report(y_test, y_pred))

            # Afficher les meilleurs paramètres
            print(f"Best parameters for {column} with {vectorizer.__class__.__name__} and {model_name}: {grid_search.best_params_}")


Erreur lors de l'ajustement du modèle LogisticRegression : 
All the 120 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
24 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\romar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\romar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\pipeline.py", line 423, in fit
    Xt = self._fit(X, y, **fit_params_steps)
         ^^^^